##### Grading Feedback Cell

# IST 718: Big Data Analytics

- Professor: Willard Williamson <wewillia@syr.edu>
- Faculty Assistant: Vidushi Mishra <vmishr01@syr.edu>
- Faculty Assistant: Pranav Kottoli Radhakrishna <pkottoli@syr.edu>
## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Code from the class text books or class provided code can be copied in its entirety.__
- There could be tests in some cells (i.e., `assert` and `np.testing.` statements). These tests (if present) are used to grade your answers. **However, the professor and FAs could use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before submitting your work, remember to check for run time errors with the following procedure:
`Kernel`$\rightarrow$`Restart and Run All`.  All runtime errors will result in a minimum penalty of half off.
- Data Bricks is the official class runtime environment so you should test your code on Data Bricks before submission.  If there is a runtime problem in the grading environment, we will try your code on Data Bricks before making a final grading decision.
- All plots shall include descriptinve title and axis labels.  Plot legends shall be included where possible.  Unless stated otherwise, plots can be made using any Python plotting package.  It is understood that spark data structures must be converted to something like numpy or pandas prior to making plots.  All required mathematical operations, filtering, selection, etc., required by a homework question shall be performed in spark prior to converting to numpy or pandas.
- Grading feedback cells are there for graders to provide feedback to students.  Don't change or remove grading feedback cells.
- Don't add or remove files from your git repo.
- Do not change file names in your repo.  This also means don't change the title of the ipython notebook.
- You are free to add additional code cells around the cells marked `your code here`.
- We reserve the right to take points off for operations that are extremely inefficient or "heavy weight".  This is a big data class and extremely inefficient operations make a big difference when scaling up to large data sets.  For example, the spark dataframe collect() method is a very heavy weight operation and should not be used unless it there is a real need for it.  An example where collect() might be needed is to get ready to make a plot after filtering a spark dataframe.
- import * is not allowed because it is considered a very bad coding practice and in some cases can result in a significant delay (which slows down the grading process) in loading imports.  For example, the statement `from sympy import *` is not allowed.  You must import the specific packages that you need.
- If you perform databricks specific operations, you MUST protect those operations in an if statement by calling the is_databricks() function provided in all homework assignments.  For example, if you use dbutils (databricks utilities), only run dbutils commands if is_databricks() returns true.  Runtime errors created in the grading environment by not protecting databricks specific commands in an if statement will result in a runtime error points deduction. 
- The graders reserve the right to deduct points for subjective things we see with your code.  For example, if we ask you to create a pandas data frame to display values from an investigation and you hard code the values, we will take points off for that.  This is only one of many different things we could find in reviewing your code.  In general, write your code like you are submitting it for a code peer review in industry.  
- Level of effort is part of our subjective grading.  Oftentimes there is a large disparity between the level of effort between students who are trying learn, and students who are trying to do the minimum possible to check off an assignment requirement.  For example, in cases where we ask for a more open ended investigation, some students put in significant effort and some students do the minimum possible to meet requirements.  In these cases, we may take points off for students who did not put in much effort as compared to students who did put in a lot of effort.  We feel that the students who did a better job deserve a better grade.  We reserve the right to invoke level of effort grading at any time.
- Only use spark, spark machine learning, spark data frames, RDD's, and map reduce to solve all problems unless instructed otherwise.
- Unless code is provided which reads data files, __you must use the get_training_filename function povided below to read data files."  Runtime errors encountered while grading caused by students not using get_training_filename will result in a minimum of half points off for the problem in question.__

In [3]:
# Do not delete or change this cell

enable_grid = True

# grading import statements
%matplotlib inline
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
# Correct Usage Example (pass ONLY the full file name):
#   file_name_to_load = get_training_filename("sms_spam.csv") # correct - pass ONLY the full file name  
#   
# Incorrect Usage Example
#   file_name_to_load = get_training_filename("/sms_spam.csv") # incorrect - pass ONLY the full file name
#   file_name_to_load = get_training_filename("sms_spam.csv/") # incorrect - pass ONLY the full file name
#   file_name_to_load = get_training_filename("c:/users/will/data/sms_spam.csv") incorrect -pass ONLY the full file name
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "dbfs:/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

# Sentiment Analysis
In this assignment, you will use the tweets.csv file to perform sentiment analysis. The tweets.csv file contains the following columns:
- target: the polarity of the tweet (0 = negative, 4 = positive)
- ids: The id of the tweet ( 2087)
- date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)
- flag: The query (lyx). If there is no query, then this value is NO_QUERY.
- user: the user that tweeted (robotickilldozr)
- text: the text of the tweet (Lyx is cool)

# Qustion 1: (10 pts)
Read tweets.csv into a spark dataframe named `tweets_df`.  Solutions that do not use `get_training_filename` will be heavily penalized.  Drop all columns except target and text.  Transform the target column such that a negative sentiment is equal to 0 and a positive sentiment is equal to 1.  Determine and print the percentage of positive and negative tweets in the dataframe such that it's easy for the graders to find and interpret your data.

In [6]:
# your code here
import pyspark.sql.functions as f

#read in the tweets.csv file
tweets_df = spark.read.csv(get_training_filename('tweets.csv'), header = True, inferSchema = True)

#drop all columns except target and text
tweets_df = tweets_df.drop("id", "date", "flag", "user")

tweets_df = tweets_df.withColumn('target', regexp_replace('target', '4', '1'))

tweets_df.groupBy('target').count().withColumn('percentage', (f.col('count')/tweets_df.count()) * 100).show()

+------+-----+----------+
target|count|percentage|
+------+-----+----------+
 1|50000| 50.0|
 0|50000| 50.0|
+------+-----+----------+

In [7]:
# grading cell do not modify
tweets_pd = tweets_df.toPandas()
display(tweets_pd.head())
print(tweets_pd.shape)

target,text
1,I LOVE @Health4UandPets u guys r the best!!
1,im meeting up with one of my besties tonight! Cant wait!! - GIRL TALK!!
1,"@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart."
1,"Being sick can be really cheap when it hurts too much to eat real food Plus, your friends make you soup"
1,@LovesBrooklyn2 he has that effect on everyone


##### Grading Feedback Cell

# Question 2: (10 pts)
Pre-process the data by creating a pipeline named `tweets_pre_proc_pipe`. Your pipeline should tokenize, remove stop words, and do a TF-IDF transformation.  Fit and execute your pipeline, and create a new dataframe named `tweets_pre_proc_df`.  Print the shape of the resulting TF-IDF data such that it's easy for the graders to find and understand as num rows x num words. Based on the shape of the TF-IDF data, would you expect a logistic regression model to overfit?

In [10]:
# your code here
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF

# create a pipeline: tokenize, remove stop words, do TF-IDF transformation

tokenizer = Tokenizer().setInputCol('text').setOutputCol('words')

stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()

sw_filter = StopWordsRemover().setStopWords(stop_words).setCaseSensitive(False).setInputCol("words").setOutputCol("filtered")

cv = CountVectorizer(minTF=1., minDF=5., vocabSize=2**17).setInputCol("filtered").setOutputCol("tf")

idf = IDF().setInputCol('tf').setOutputCol('tf-idf')

tweets_pre_proc_pipe = Pipeline(stages = [tokenizer, sw_filter, cv, idf]).fit(tweets_df)

# create a new dataframe named: tweets_pre_proc_df

tweets_pre_proc_df = tweets_pre_proc_pipe.transform(tweets_df)

# print the shape of the resulting TF-IDF data

print(tweets_pre_proc_df.toPandas().shape)

(100000, 6)

In [11]:
# grading cell do not modify
display(tweets_pre_proc_df.toPandas().head())

target,text,words,filtered,tf,tf-idf
1,I LOVE @Health4UandPets u guys r the best!!,"List(i, love, @health4uandpets, u, guys, r, the, best!!)","List(love, @health4uandpets, u, guys, r, best!!)","List(0, 13693, List(11, 19, 145, 259, 12280), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 13693, List(11, 19, 145, 259, 12280), List(3.4308421867418466, 3.869452557843028, 5.184998681191034, 5.657863542717801, 9.721175995692173))"
1,im meeting up with one of my besties tonight! Cant wait!! - GIRL TALK!!,"List(im, meeting, up, with, one, of, my, besties, tonight!, cant, wait!!, , -, girl, talk!!)","List(im, meeting, besties, tonight!, wait!!, , -, girl, talk!!)","List(0, 13693, List(0, 5, 15, 246, 493, 638, 5993, 6784), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 13693, List(0, 5, 15, 246, 493, 638, 5993, 6784), List(1.1265280578718189, 3.1835186809809097, 3.667911047678745, 5.572764212199798, 6.2502452760153435, 6.482497543527793, 8.947986107458693, 9.02802881513223))"
1,"@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart.","List(@darealsunisakim, thanks, for, the, twitter, add,, sunisa!, i, got, to, meet, you, once, at, a, hin, show, here, in, the, dc, area, and, you, were, a, sweetheart.)","List(@darealsunisakim, thanks, twitter, add,, sunisa!, got, meet, hin, dc, area, sweetheart.)","List(0, 13693, List(9, 29, 31, 290, 1888, 2933, 12192), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 13693, List(9, 29, 31, 290, 1888, 2933, 12192), List(3.328421711886508, 3.979776657464666, 4.004696690241565, 5.738383919375821, 7.6417344540123375, 8.111738083258073, 9.721175995692173))"
1,"Being sick can be really cheap when it hurts too much to eat real food Plus, your friends make you soup","List(being, sick, can, be, really, cheap, when, it, hurts, too, much, to, eat, real, food, , plus,, your, friends, make, you, soup)","List(sick, really, cheap, hurts, eat, real, food, , plus,, friends, make, soup)","List(0, 13693, List(0, 14, 41, 95, 116, 229, 265, 300, 302, 1640, 1990, 4828), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 13693, List(0, 14, 41, 95, 116, 229, 265, 300, 302, 1640, 1990, 4828), List(1.1265280578718189, 3.630997975648159, 4.248205286990362, 4.933684252910128, 5.057736901580107, 5.51897403761366, 5.68988956943721, 5.7893503629678476, 5.760362826094596, 7.505602279687758, 7.684294068431134, 8.679722120864012))"
1,@LovesBrooklyn2 he has that effect on everyone,"List(@lovesbrooklyn2, he, has, that, effect, on, everyone)","List(@lovesbrooklyn2, effect)","List(0, 13693, List(2808), List(1.0))","List(0, 13693, List(2808), List(8.078948260435082))"


Your explanation here:

##### Grading Feedback Cell

# Question 3: (10 pts)
Since IDF considers a word's frequency across all documents in a corpus, you can use IDF as a form of inference.  Examine the documentation for the spark ML object that you used to create TF-IDF scores and learn how to extract the IDF scores for words in the corpus.  Create a pandas dataframe containing the 5 most important IDF scores named `most_imp_idf`.  Create another pandas dataframe containing the 5 least important IDF scores named `least_imp_idf`.  Each dataframe shall have 2 columns named `word` and `idf_score`.  Explain in words your interpretation of what the IDF scores mean.

In [15]:
# your code here

In [16]:
# grading cell do not modify
display(most_imp_idf)
display(least_imp_idf)

Your explanation here:

##### Grading Feedback Cell

# Question 4: (10 pts)
Create a new recursive pipeline named `lr_pipe` which starts with `tweets_pre_proc_pipe` and adds a logistic regression model using default hyper parameters.  Fit lr_pipe using `tweets_df`.  Score the model using ROC AUC.  Report the resulting AUC such that it is easy for graders to find and interpret.

In [20]:
# your code here

##### Grading Feedback Cell

# Question 5: (10 pts)
Create 2 pandas dataframes named `lr_pipe_df_neg` and `lr_pipe_df_pos`which contain 2 colunms: `word` and `score`.  Load the 2 dataframes with the top 10 words and logistic regression coefficients that contribute the most to negative and positive sentiments respectively. Analyze the 2 dataframes and describe if the words make sense.  Do the words look like they are really negative and positive?

In [23]:
# your code here

In [24]:
# grading cell - do not modify
display(lr_pipe_df_neg)
display(lr_pipe_df_pos)

Your explanation here:

##### Grading Feedback Cell

# Question 6a: (5 pts)
The goal of this question is to try to improve the score from question 4 using a regularization grid search on a new pipeline named `lr_pipe_1`. lr_pipe_1 is the same as lr_pipe above but we would like you to create a new pipe for grading purposes only.  I'm not sure if it's possible to increase the score or not.  You will be graded on level of effort to increase the score in relation to other students in the class.  All of your grid search code should be inside the `if enable_grid` statement in the cell below.  The enable_grid boolean is set to true in a grading cell above.  If any of the grid search code executes outside of the if statement, you will not get full credit for the question.  We want the ability to turn off the grid search during grading.<br>

In [28]:
# your grid search (and only your grid search) code here
if enable_grid:
    # your grid search code here
    pass

##### Grading feedback cell

# Question 6b (5 pts)
Build a new pipeline named `lr_pipe_2` which uses the optimized model parameters from the grid search in question 6a above (the best model).  Create 2 variables named alpha and lambda and assign to them the best alpha and lambda produced by the grid search by hard coding the values. Fit and transform lr_pipe_2.  Compare AUC scores between lr_pipe_2 with lr_pipe in question 4.  Create a pandas dataframe named `comapre_1_df` which encapsulates the comparison data.  comapre_1_df Shall have 2 columns: `model_name` and `auc_score`.

In [31]:
# your optimized model code here
# example
# alpha = 0.1
# lambda = 0.1

# lr_pipe_2 code here

In [32]:
# grading cell - do not modify
display(comapre_1_df)

##### Grading Feedback Cell

# Question 7 (10 pts)
Perform inference on lr_pipe_2.  Write code to report how many words were eliminated from the best model in question 6b above (if any) as compared to the model in question 4 above.  Make sure your output is easy for the graders to find and interpret.

In [35]:
# your code here

##### Grading Feedback Cell

# Question 8 (10 pts)
Perform the same inference analysis that you did in question 5 but name the data frames `lr_pipe_df_neg_1` and `lr_pipe_df_pos_1`.  Compare the word importance results with the results in question 5.

In [38]:
# your code here

In [39]:
# grading cell - do not modify
display(lr_pipe_df_neg_1)
display(lr_pipe_df_pos_1)

##### Grading Feedback Cell

Your explanation here:

##### Grading Feedback Cell

# Question 9 (10 pts)
Create a receiver operating characteristic (ROC) plot for the best model in question 6.  Briefly describe in words the high level steps needed to build a ROC curve as outlined in lecture.  Convince me you understand the high level steps needed to make a ROC curve.

In [44]:
# your code here

Your explanation here:

##### Grading Feedback Cell

# Question 10 (10 pts)
Learn about [precision/recall](https://en.wikipedia.org/wiki/Precision_and_recall) curves. Using the logistic regression summary object contained in the linear regression object within lr_pipe_2, create a precision recall plot. Similar to the `roc` object which is available in the logistic regression summary, there is a `pr` object which can be used to help create a precision / recall curve.  Note that the precision recall curve is built using the same high level methodology as the ROC curve, but using different metrics calculated from the confusion matrix.  If you understand how a ROC curve is built, you understand how a precision / recall curve is built.  Compare and contrast the differences between precision / recall and ROC curves.  What axis is common and what axis is different between the 2 curves?

In [48]:
# your code here

Your explanation here:

##### Grading Feedback Cell